In [30]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import matplotlib 
## import pytorch_lightning as pl
import torch 

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


In [31]:

## Python = 3.9


In [32]:

pd.options.display.max_columns = None
pd.options.display.max_rows    = 30



## The 5998 merged file


In [33]:

for_RNN_data_CIVS = pd.read_csv('merged_seq_5998_CIVS.csv')


In [34]:

for_RNN_data_CIVS.head(5)


,pd_index,date_f1,CNUM,LNUM_f1,SI_f1,S_f1,MN_f1,P_f1,CU_f1,NI_f1,CR_f1,MO_f1,TI_f1,HM_Temp_f1,ProbeSi_f1,Time_f1,Timestamp_f1,Compliant_eq_1_f1,C_perce_sat_f1,Before_After_f1,Good_bad_f1,date,Slag_Time_f2,Open_Time_f2,Close_Time_f2,Taphole_f2,Pipe_burn_hole_f2,Taphole_Length_f2,Clay_box_f2,Clay_Close_hole_ft3_f2,Clay_temp_f2,Drill_bit_size_f2,Drill_shafts_used_f2,Cast_Avg_Temp_f2,Cast_Avg_Si_f2,Cast_Avg_S_f2,Cast_Avg_Mn_f2,Cast_Avg_P_f2,Cast_Avg_Cu_f2,Cast_Avg_Ni_f2,Cast_Avg_Cr_f2,Cast_Avg_Mo_f2,Cast_Avg_Ti_f2,Slag_Fe_f2,Slag_SiO2_f2,Slag_Al2O3_f2,Slag_CaO_f2,Slag_MgO_f2,Slag_S_f2,Slag_K2O_f2,Slag_B_SiO2_f2,Slag_TiO2_f2
0,0,44409,2809,182,1.152,0.042,0.577,0.077,0.003,0.020,0.052,0.002,0.036,2677,NaN,0.057639,44409.05764,NaN,4.692156,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2,0,11.4,602,2.75,165,2.12,1,2714,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
1,1,44409,2809,171,0.932,0.053,0.521,0.071,0.003,0.015,0.051,0.002,0.027,2729,NaN,0.086111,44409.08611,1.0,4.830898,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2,0,11.4,602,2.75,165,2.12,1,2714,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
2,2,44409,2809,122,1.059,0.043,0.540,0.076,0.003,0.020,0.051,0.002,0.032,2736,NaN,0.131250,44409.13125,NaN,4.804690,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2,0,11.4,602,2.75,165,2.12,1,2714,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
3,3,44409,2810,147,0.917,0.059,0.478,0.072,0.003,0.017,0.049,0.002,0.024,2557,NaN,0.145833,44409.14583,1.0,4.587464,B,0.0,8/1/2021 0:00,03:45,03:20,06:01,3,0,11.5,596,4.00,160,2.25,1,2677,0.906,0.057,0.488,0.072,0.003,0.016,0.049,0.002,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34
4,4,44409,2810,190,1.023,0.054,0.508,0.072,0.003,0.012,0.048,0.002,0.029,2705,NaN,0.170139,44409.17014,NaN,4.768184,B,0.0,8/1/2021 0:00,03:45,03:20,06:01,3,0,11.5,596,4.00,160,2.25,1,2677,0.906,0.057,0.488,0.072,0.003,0.016,0.049,0.002,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34


In [35]:

for_RNN_data_CIVS.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pd_index                5998 non-null   int64  
 1   date_f1                 5998 non-null   int64  
 2   CNUM                    5998 non-null   int64  
 3   LNUM_f1                 5998 non-null   int64  
 4   SI_f1                   5489 non-null   float64
 5   S_f1                    5479 non-null   float64
 6   MN_f1                   5998 non-null   float64
 7   P_f1                    5998 non-null   float64
 8   CU_f1                   5998 non-null   float64
 9   NI_f1                   5998 non-null   float64
 10  CR_f1                   5998 non-null   float64
 11  MO_f1                   5998 non-null   float64
 12  TI_f1                   5998 non-null   float64
 13  HM_Temp_f1              5998 non-null   int64  
 14  ProbeSi_f1              6 non-null      

In [36]:

for_RNN_data_CIVS


,pd_index,date_f1,CNUM,LNUM_f1,SI_f1,S_f1,MN_f1,P_f1,CU_f1,NI_f1,CR_f1,MO_f1,TI_f1,HM_Temp_f1,ProbeSi_f1,Time_f1,Timestamp_f1,Compliant_eq_1_f1,C_perce_sat_f1,Before_After_f1,Good_bad_f1,date,Slag_Time_f2,Open_Time_f2,Close_Time_f2,Taphole_f2,Pipe_burn_hole_f2,Taphole_Length_f2,Clay_box_f2,Clay_Close_hole_ft3_f2,Clay_temp_f2,Drill_bit_size_f2,Drill_shafts_used_f2,Cast_Avg_Temp_f2,Cast_Avg_Si_f2,Cast_Avg_S_f2,Cast_Avg_Mn_f2,Cast_Avg_P_f2,Cast_Avg_Cu_f2,Cast_Avg_Ni_f2,Cast_Avg_Cr_f2,Cast_Avg_Mo_f2,Cast_Avg_Ti_f2,Slag_Fe_f2,Slag_SiO2_f2,Slag_Al2O3_f2,Slag_CaO_f2,Slag_MgO_f2,Slag_S_f2,Slag_K2O_f2,Slag_B_SiO2_f2,Slag_TiO2_f2
0,0,44409,2809,182,1.152,0.042,0.577,0.077,0.003,0.020,0.052,0.002,0.036,2677,NaN,0.057639,44409.05764,NaN,4.692156,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2,0,11.4,602,2.75,165,2.12,1,2714,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
1,1,44409,2809,171,0.932,0.053,0.521,0.071,0.003,0.015,0.051,0.002,0.027,2729,NaN,0.086111,44409.08611,1.0,4.830898,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2,0,11.4,602,2.75,165,2.12,1,2714,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
2,2,44409,2809,122,1.059,0.043,0.540,0.076,0.003,0.020,0.051,0.002,0.032,2736,NaN,0.131250,44409.13125,NaN,4.804690,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2,0,11.4,602,2.75,165,2.12,1,2714,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
3,3,44409,2810,147,0.917,0.059,0.478,0.072,0.003,0.017,0.049,0.002,0.024,2557,NaN,0.145833,44409.14583,1.0,4.587464,B,0.0,8/1/2021 0:00,03:45,03:20,06:01,3,0,11.5,596,4.00,160,2.25,1,2677,0.906,0.057,0.488,0.072,0.003,0.016,0.049,0.002,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34
4,4,44409,2810,190,1.023,0.054,0.508,0.072,0.003,0.012,0.048,0.002,0.029,2705,NaN,0.170139,44409.17014,NaN,4.768184,B,0.0,8/1/2021 0:00,03:45,03:20,06:01,3,0,11.5,596,4.00,160,2.25,1,2677,0.906,0.057,0.488,0.072,0.003,0.016,0.049,0.002,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,5993,44592,379,120,0.795,0.019,0.397,0.054,0.003,0.003,0.041,0.002,0.057,2765,NaN,0.838889,44592.83889,1.0,4.940506,NaN,NaN,1/31/2022 0:00,19:20,19:12,20:54,1,0,9.6,59,5.00,176,2.13,1,2739,0.721,0.021,0.400,0.055,0.003,0.003,0.042,0.002,0.052,0.51,39.639999,8.66,41.150002,8.27,0.93,0.63,1.25,0.44
5994,5994,44592,379,121,NaN,NaN,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2772,NaN,0.870833,44592.87083,NaN,5.211540,NaN,NaN,1/31/2022 0:00,19:20,19:12,20:54,1,0,9.6,59,5.00,176,2.13,1,2739,0.721,0.021,0.400,0.055,0.003,0.003,0.042,0.002,0.052,0.51,39.639999,8.66,41.150002,8.27,0.93,0.63,1.25,0.44
5995,5995,44592,380,163,0.598,0.025,0.383,0.054,0.003,0.003,0.039,0.002,0.042,2746,NaN,0.895833,44592.89583,1.0,4.973084,NaN,NaN,1/31/2022 0:00,21:20,20:52,22:43,2,0,11.3,6,3.00,160,2.25,1,2747,0.584,0.027,0.360,0.053,0.003,0.003,0.038,0.002,0.041,0.32,39.090000,8.67,40.560001,7.86,0.99,0.56,1.24,0.42
5996,5996,44592,380,146,0.608,0.030,0.351,0.052,0.003,0.003,0.038,0.002,0.044,2748,NaN,0.902778,44592.90278,1.0,4.970318,NaN,NaN,1/31/2022 0:00,21:20,20:52,22:43,2,0,11.3,6,3.00,160,2.25,1,2747,0.584,0.027,0.360,0.053,0.003,0.003,0.038,0.002,0.041,0.32,39.090000,8.67,40.560001,7.86,0.99,0.56,1.24,0.42


In [37]:

for_RNN_data_CIVS['date'][2000:2400]


2000     9/30/2021 0:00
2001     9/30/2021 0:00
2002     9/30/2021 0:00
2003     9/30/2021 0:00
2004     9/30/2021 0:00
             ...       
2395    10/10/2021 0:00
2396    10/10/2021 0:00
2397    10/10/2021 0:00
2398    10/10/2021 0:00
2399    10/10/2021 0:00
Name: date, Length: 400, dtype: object


## Add large file of ~137 MB


In [38]:

MinByMin_137MB_data_CIVS = pd.read_csv('CIVS_137_MB_MinbyMin.csv')


In [39]:

MinByMin_137MB_data_CIVS.head(5)


,date,THEORETICAL_TONNAGE_f3,Coke_Rate_f3,HOT_BLAST_TMP_NS_f3,WINDRATE_f3,SNORT_VALVE_POSITION_f3,COLD_BLAST_MOISTURE_f3,HOST_BLAST_MOISTURE_f3,HIGH_PURITY_OXYGEN_f3,COAL_FLOW_f3,NAT_GAS _INJECTION_f3,HOT_BLAST_PRESSURE_f3,TOP_PRESS_f3,TOP_TEMP_HIGHEST_f3,Top_Gas_ETACO_f3,Selec_Top_Gas_CO_f3,Selec_Top_Gas_CO2_f3,Selec_Top_Gas_H2_f3,Selec_Top_Gas_N2_f3,SILICON_f3,SULFUR_f3,MANGANESE_f3,PHOSPHOROUS_f3,HOT_METAL_TEMP_f3,Slag_CaO_day_avg_f3,Slag_MgO_day_avg_f3,Slag_SiO2_f3,Slag_Al2O3_day_avg_f3,#14 WEST STOCKROD LEVEL,#14 EAST STOCKROD LEVEL,#14 RADAR ROD NORTH LEVEL,#14 RADAR ROD SOUTH LEVEL,Charges_per_hour_f3,BF14 material 1 weight,BF14 extra coke weight,BF14 material 2 weight,BF14 material 2 weight.1,BF14 material 3 weight,BF14 material 3 weight.1,BF14 material 4 weight,BF14 material 4 weight.1,BF14 material 5 weight,BF14 material 5 weight.1,BF14 material 6 weight,BF14 material 6 weight.1,BF14 material 7 weight,BF14 material 7 weight.1,BF14 material 8 weight,BF14 material 8 weight.1,BF14 material 9 weight,BF14 material 9 weight.1,BF14 material 10 weight,BF14 material 10 weight.1,NE_Uptake_f3,SE_Uptake_f3,NW_Uptake_f3,SW_Uptake_f3
0,8/1/21 0:00,6006,992,1999.604384,179.7496382,98.33429718,4.997584786,8.084516915,2.510679501,0,4034.234339,34.35759044,17.39832633,245.3351088,0.458302918,22.32206753,18.72929568,3.559209619,54.25013897,0.776,0.052,0.51,0.068,2736.960938,37.830002,10.577999,38.029511,7.084,8.720903871,-0.001068148,8.4555001,8.706065964,4.172247,226085.1667,309.166667,653422.0,15730100,103426.0,2756060,0,0,0,0,0.0,0,31782.33333,848140,0,0,21118.33333,562140,9022.666667,212980,158.478922,245.6341754,145.3965382,220.7173486
1,8/1/21 0:01,6006,992,2000.288837,179.7659446,98.33429718,5.000530552,8.079545175,2.489380854,0,4024.947376,34.32882112,17.48348565,251.1982621,0.458276544,22.3353289,18.7313195,3.559234369,54.19425672,0.776,0.052,0.51,0.068,2736.960938,37.830002,10.577999,38.029511,7.084,3.973097455,-0.001068148,8.617395605,8.988543235,4.166692,226655.5000,927.500000,651306.0,15730100,103438.0,2756060,0,0,0,0,0.0,0,31787.00000,848140,0,0,21115.00000,562140,8988.000000,212980,159.637733,252.4592171,145.5708951,226.3606003
2,8/1/21 0:02,6006,992,1999.242862,179.7822511,98.33429718,5.003476318,8.074573435,2.489036886,0,4011.601682,34.3000518,17.26035822,242.6096693,0.45825017,22.34859026,18.73334332,3.559259119,54.13837447,0.776,0.052,0.51,0.068,2736.960938,37.830002,10.577999,38.029511,7.084,0.158668744,-0.001068148,8.654925812,9.076584724,4.166692,227225.8333,1545.833333,649190.0,15730100,103450.0,2756060,0,0,0,0,0.0,0,31791.66667,848140,0,0,21111.66667,562140,8953.333333,212980,160.796544,252.6457359,145.745252,230.2975397
3,8/1/21 0:03,6006,992,1999.884442,179.7984904,98.33429718,5.006422084,8.069601695,2.49686719,0,4012.024633,34.27128248,17.21260561,232.798103,0.458223796,22.36185163,18.73536714,3.559283869,54.0827965,0.776,0.052,0.51,0.068,2736.960938,37.830002,10.577999,38.029511,7.084,3.247876226,-0.001068148,8.932542874,9.255977372,4.166692,227796.1667,2164.166667,647074.0,15730100,103462.0,2756060,0,0,0,0,0.0,0,31796.33333,848140,0,0,21108.33333,562140,8918.666667,212980,161.955355,247.987563,145.9196089,225.2202915
4,8/1/21 0:04,6006,992,1999.849729,179.7844299,98.33429718,5.009367851,8.064629955,2.501961434,0,4000.467204,34.24251316,17.40503504,224.0119417,0.458197422,22.375113,18.73739096,3.559308618,54.06377896,0.776,0.052,0.51,0.068,2736.960938,37.830002,10.577999,38.029511,7.084,0.040138009,-0.001068148,9.161944934,9.261321576,4.166692,228366.5000,2782.500000,644958.0,15730100,103474.0,2756060,0,0,0,0,0.0,0,31801.00000,848140,0,0,21105.00000,562140,8884.000000,212980,163.1027365,226.923719,146.0939658,218.8276934


In [40]:

MinByMin_137MB_data_CIVS.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265020 entries, 0 to 265019
Data columns (total 57 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date                       265020 non-null  object 
 1   THEORETICAL_TONNAGE_f3     265020 non-null  int64  
 2   Coke_Rate_f3               265020 non-null  int64  
 3   HOT_BLAST_TMP_NS_f3        265020 non-null  object 
 4   WINDRATE_f3                265020 non-null  object 
 5   SNORT_VALVE_POSITION_f3    265020 non-null  object 
 6   COLD_BLAST_MOISTURE_f3     265020 non-null  object 
 7   HOST_BLAST_MOISTURE_f3     265020 non-null  object 
 8   HIGH_PURITY_OXYGEN_f3      265020 non-null  object 
 9   COAL_FLOW_f3               265020 non-null  object 
 10  NAT_GAS _INJECTION_f3      265020 non-null  object 
 11  HOT_BLAST_PRESSURE_f3      265020 non-null  object 
 12  TOP_PRESS_f3               265020 non-null  object 
 13  TOP_TEMP_HIGHEST_f3        26

In [41]:

MinByMin_137MB_data_CIVS


,date,THEORETICAL_TONNAGE_f3,Coke_Rate_f3,HOT_BLAST_TMP_NS_f3,WINDRATE_f3,SNORT_VALVE_POSITION_f3,COLD_BLAST_MOISTURE_f3,HOST_BLAST_MOISTURE_f3,HIGH_PURITY_OXYGEN_f3,COAL_FLOW_f3,NAT_GAS _INJECTION_f3,HOT_BLAST_PRESSURE_f3,TOP_PRESS_f3,TOP_TEMP_HIGHEST_f3,Top_Gas_ETACO_f3,Selec_Top_Gas_CO_f3,Selec_Top_Gas_CO2_f3,Selec_Top_Gas_H2_f3,Selec_Top_Gas_N2_f3,SILICON_f3,SULFUR_f3,MANGANESE_f3,PHOSPHOROUS_f3,HOT_METAL_TEMP_f3,Slag_CaO_day_avg_f3,Slag_MgO_day_avg_f3,Slag_SiO2_f3,Slag_Al2O3_day_avg_f3,#14 WEST STOCKROD LEVEL,#14 EAST STOCKROD LEVEL,#14 RADAR ROD NORTH LEVEL,#14 RADAR ROD SOUTH LEVEL,Charges_per_hour_f3,BF14 material 1 weight,BF14 extra coke weight,BF14 material 2 weight,BF14 material 2 weight.1,BF14 material 3 weight,BF14 material 3 weight.1,BF14 material 4 weight,BF14 material 4 weight.1,BF14 material 5 weight,BF14 material 5 weight.1,BF14 material 6 weight,BF14 material 6 weight.1,BF14 material 7 weight,BF14 material 7 weight.1,BF14 material 8 weight,BF14 material 8 weight.1,BF14 material 9 weight,BF14 material 9 weight.1,BF14 material 10 weight,BF14 material 10 weight.1,NE_Uptake_f3,SE_Uptake_f3,NW_Uptake_f3,SW_Uptake_f3
0,8/1/21 0:00,6006,992,1999.604384,179.7496382,98.33429718,4.997584786,8.084516915,2.510679501,0,4034.234339,34.35759044,17.39832633,245.3351088,0.458302918,22.32206753,18.72929568,3.559209619,54.25013897,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,8.720903871,-0.001068148,8.4555001,8.706065964,4.172247,226085.1667,309.166667,653422.0000,15730100,103426.00000,2756060,0,0,0,0,0.000000,0,31782.33333,848140,0,0,21118.33333,562140,9022.666667,212980,158.478922,245.6341754,145.3965382,220.7173486
1,8/1/21 0:01,6006,992,2000.288837,179.7659446,98.33429718,5.000530552,8.079545175,2.489380854,0,4024.947376,34.32882112,17.48348565,251.1982621,0.458276544,22.3353289,18.7313195,3.559234369,54.19425672,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,3.973097455,-0.001068148,8.617395605,8.988543235,4.166692,226655.5000,927.500000,651306.0000,15730100,103438.00000,2756060,0,0,0,0,0.000000,0,31787.00000,848140,0,0,21115.00000,562140,8988.000000,212980,159.637733,252.4592171,145.5708951,226.3606003
2,8/1/21 0:02,6006,992,1999.242862,179.7822511,98.33429718,5.003476318,8.074573435,2.489036886,0,4011.601682,34.3000518,17.26035822,242.6096693,0.45825017,22.34859026,18.73334332,3.559259119,54.13837447,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,0.158668744,-0.001068148,8.654925812,9.076584724,4.166692,227225.8333,1545.833333,649190.0000,15730100,103450.00000,2756060,0,0,0,0,0.000000,0,31791.66667,848140,0,0,21111.66667,562140,8953.333333,212980,160.796544,252.6457359,145.745252,230.2975397
3,8/1/21 0:03,6006,992,1999.884442,179.7984904,98.33429718,5.006422084,8.069601695,2.49686719,0,4012.024633,34.27128248,17.21260561,232.798103,0.458223796,22.36185163,18.73536714,3.559283869,54.0827965,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,3.247876226,-0.001068148,8.932542874,9.255977372,4.166692,227796.1667,2164.166667,647074.0000,15730100,103462.00000,2756060,0,0,0,0,0.000000,0,31796.33333,848140,0,0,21108.33333,562140,8918.666667,212980,161.955355,247.987563,145.9196089,225.2202915
4,8/1/21 0:04,6006,992,1999.849729,179.7844299,98.33429718,5.009367851,8.064629955,2.501961434,0,4000.467204,34.24251316,17.40503504,224.0119417,0.458197422,22.375113,18.73739096,3.559308618,54.06377896,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,0.040138009,-0.001068148,9.161944934,9.261321576,4.166692,228366.5000,2782.500000,644958.0000,15730100,103474.00000,2756060,0,0,0,0,0.000000,0,31801.00000,848140,0,0,21105.00000,562140,8884.000000,212980,163.1027365,226.923719,146.0939658,218.8276934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [42]:

MinByMin_137MB_data_CIVS['date'][1000:1400]


1000    8/1/21 16:40
1001    8/1/21 16:41
1002    8/1/21 16:42
1003    8/1/21 16:43
1004    8/1/21 16:44
            ...     
1395    8/1/21 23:15
1396    8/1/21 23:16
1397    8/1/21 23:17
1398    8/1/21 23:18
1399    8/1/21 23:19
Name: date, Length: 400, dtype: object


## Iterate through pandas df


In [56]:

MinByMin_137MB_data_CIVS['date_from_6000f'] = 'abc'


In [57]:

MinByMin_137MB_data_CIVS


,date,THEORETICAL_TONNAGE_f3,Coke_Rate_f3,HOT_BLAST_TMP_NS_f3,WINDRATE_f3,SNORT_VALVE_POSITION_f3,COLD_BLAST_MOISTURE_f3,HOST_BLAST_MOISTURE_f3,HIGH_PURITY_OXYGEN_f3,COAL_FLOW_f3,NAT_GAS _INJECTION_f3,HOT_BLAST_PRESSURE_f3,TOP_PRESS_f3,TOP_TEMP_HIGHEST_f3,Top_Gas_ETACO_f3,Selec_Top_Gas_CO_f3,Selec_Top_Gas_CO2_f3,Selec_Top_Gas_H2_f3,Selec_Top_Gas_N2_f3,SILICON_f3,SULFUR_f3,MANGANESE_f3,PHOSPHOROUS_f3,HOT_METAL_TEMP_f3,Slag_CaO_day_avg_f3,Slag_MgO_day_avg_f3,Slag_SiO2_f3,Slag_Al2O3_day_avg_f3,#14 WEST STOCKROD LEVEL,#14 EAST STOCKROD LEVEL,#14 RADAR ROD NORTH LEVEL,#14 RADAR ROD SOUTH LEVEL,Charges_per_hour_f3,BF14 material 1 weight,BF14 extra coke weight,BF14 material 2 weight,BF14 material 2 weight.1,BF14 material 3 weight,BF14 material 3 weight.1,BF14 material 4 weight,BF14 material 4 weight.1,BF14 material 5 weight,BF14 material 5 weight.1,BF14 material 6 weight,BF14 material 6 weight.1,BF14 material 7 weight,BF14 material 7 weight.1,BF14 material 8 weight,BF14 material 8 weight.1,BF14 material 9 weight,BF14 material 9 weight.1,BF14 material 10 weight,BF14 material 10 weight.1,NE_Uptake_f3,SE_Uptake_f3,NW_Uptake_f3,SW_Uptake_f3,date_from_6000f
0,8/1/21 0:00,6006,992,1999.604384,179.7496382,98.33429718,4.997584786,8.084516915,2.510679501,0,4034.234339,34.35759044,17.39832633,245.3351088,0.458302918,22.32206753,18.72929568,3.559209619,54.25013897,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,8.720903871,-0.001068148,8.4555001,8.706065964,4.172247,226085.1667,309.166667,653422.0000,15730100,103426.00000,2756060,0,0,0,0,0.000000,0,31782.33333,848140,0,0,21118.33333,562140,9022.666667,212980,158.478922,245.6341754,145.3965382,220.7173486,abc
1,8/1/21 0:01,6006,992,2000.288837,179.7659446,98.33429718,5.000530552,8.079545175,2.489380854,0,4024.947376,34.32882112,17.48348565,251.1982621,0.458276544,22.3353289,18.7313195,3.559234369,54.19425672,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,3.973097455,-0.001068148,8.617395605,8.988543235,4.166692,226655.5000,927.500000,651306.0000,15730100,103438.00000,2756060,0,0,0,0,0.000000,0,31787.00000,848140,0,0,21115.00000,562140,8988.000000,212980,159.637733,252.4592171,145.5708951,226.3606003,abc
2,8/1/21 0:02,6006,992,1999.242862,179.7822511,98.33429718,5.003476318,8.074573435,2.489036886,0,4011.601682,34.3000518,17.26035822,242.6096693,0.45825017,22.34859026,18.73334332,3.559259119,54.13837447,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,0.158668744,-0.001068148,8.654925812,9.076584724,4.166692,227225.8333,1545.833333,649190.0000,15730100,103450.00000,2756060,0,0,0,0,0.000000,0,31791.66667,848140,0,0,21111.66667,562140,8953.333333,212980,160.796544,252.6457359,145.745252,230.2975397,abc
3,8/1/21 0:03,6006,992,1999.884442,179.7984904,98.33429718,5.006422084,8.069601695,2.49686719,0,4012.024633,34.27128248,17.21260561,232.798103,0.458223796,22.36185163,18.73536714,3.559283869,54.0827965,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,3.247876226,-0.001068148,8.932542874,9.255977372,4.166692,227796.1667,2164.166667,647074.0000,15730100,103462.00000,2756060,0,0,0,0,0.000000,0,31796.33333,848140,0,0,21108.33333,562140,8918.666667,212980,161.955355,247.987563,145.9196089,225.2202915,abc
4,8/1/21 0:04,6006,992,1999.849729,179.7844299,98.33429718,5.009367851,8.064629955,2.501961434,0,4000.467204,34.24251316,17.40503504,224.0119417,0.458197422,22.375113,18.73739096,3.559308618,54.06377896,0.776,0.052,0.510,0.068,2736.960938,37.830002,10.577999,38.029511,7.084000,0.040138009,-0.001068148,9.161944934,9.261321576,4.166692,228366.5000,2782.500000,644958.0000,15730100,103474.00000,2756060,0,0,0,0,0.000000,0,31801.00000,848140,0,0,21105.00000,562140,8884.000000,212980,163.1027365,226.923719,146.0939658,218.8276934,abc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [55]:



list_dates_6000f = for_RNN_data_CIVS['date'].tolist()
list_dates_6000f


['8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/1/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021 0:00',
 '8/2/2021

In [68]:

for_RNN_data_CIVS['temp13'] = 1

result = for_RNN_data_CIVS.groupby("date", sort=False)['temp13'].count()


result


date
8/1/2021 0:00     35
8/2/2021 0:00     35
8/3/2021 0:00     36
8/4/2021 0:00     33
8/5/2021 0:00     41
                  ..
1/27/2022 0:00    25
1/28/2022 0:00    42
1/29/2022 0:00    42
1/30/2022 0:00    46
1/31/2022 0:00    35
Name: temp13, Length: 171, dtype: int64

In [71]:

MinByMin_137MB_data_CIVS['text_new'] = MinByMin_137MB_data_CIVS['date'].str.split(' ').str[0]

MinByMin_137MB_data_CIVS['temp13'] = 1

result1 = MinByMin_137MB_data_CIVS.groupby("text_new", sort=False)['temp13'].count()


result1


text_new
8/1/21     1440
8/2/21     1440
8/3/21     1440
8/4/21     1440
8/5/21     1440
           ... 
1/27/22    1440
1/28/22    1440
1/29/22    1440
1/30/22    1440
1/31/22    1440
Name: temp13, Length: 184, dtype: int64

In [61]:

for index, row in MinByMin_137MB_data_CIVS.iterrows():
    print( row["date"] )
    input()


8/1/21 0:00


8/1/21 0:01


8/1/21 0:02


8/1/21 0:03


8/1/21 0:04


...

8/1/21 2:29


8/1/21 2:30


8/1/21 2:31


8/1/21 2:32


KeyboardInterrupt: Interrupted by user

In [62]:

for index, row in for_RNN_data_CIVS.iterrows():
    print( row["date"] )
    input()


8/1/2021 0:00


8/1/2021 0:00


8/1/2021 0:00


8/1/2021 0:00


8/1/2021 0:00


...

8/2/2021 0:00


8/2/2021 0:00


8/2/2021 0:00


8/2/2021 0:00


KeyboardInterrupt: Interrupted by user


## Merge both pandas frames


In [43]:

### result = df.groupby('Date', sort=False)['Rating'].mean().reset_index()


### df.rename(columns={"A": "a", "B": "c"})


In [44]:

## list_cols_5889 = for_RNN_data_CIVS.columns.tolist()
## list_cols_5889


In [45]:

## list_cols_137_MB = MinByMin_137MB_data_CIVS.columns.tolist()
## list_cols_137_MB


In [46]:

merged_data = pd.merge(for_RNN_data_CIVS , MinByMin_137MB_data_CIVS, on='date', how='outer')


In [47]:

merged_data


,pd_index,date_f1,CNUM,LNUM_f1,SI_f1,S_f1,MN_f1,P_f1,CU_f1,NI_f1,CR_f1,MO_f1,TI_f1,HM_Temp_f1,ProbeSi_f1,Time_f1,Timestamp_f1,Compliant_eq_1_f1,C_perce_sat_f1,Before_After_f1,Good_bad_f1,date,Slag_Time_f2,Open_Time_f2,Close_Time_f2,Taphole_f2,Pipe_burn_hole_f2,Taphole_Length_f2,Clay_box_f2,Clay_Close_hole_ft3_f2,Clay_temp_f2,Drill_bit_size_f2,Drill_shafts_used_f2,Cast_Avg_Temp_f2,Cast_Avg_Si_f2,Cast_Avg_S_f2,Cast_Avg_Mn_f2,Cast_Avg_P_f2,Cast_Avg_Cu_f2,Cast_Avg_Ni_f2,Cast_Avg_Cr_f2,Cast_Avg_Mo_f2,Cast_Avg_Ti_f2,Slag_Fe_f2,Slag_SiO2_f2,Slag_Al2O3_f2,Slag_CaO_f2,Slag_MgO_f2,Slag_S_f2,Slag_K2O_f2,Slag_B_SiO2_f2,Slag_TiO2_f2,THEORETICAL_TONNAGE_f3,Coke_Rate_f3,HOT_BLAST_TMP_NS_f3,WINDRATE_f3,SNORT_VALVE_POSITION_f3,COLD_BLAST_MOISTURE_f3,HOST_BLAST_MOISTURE_f3,HIGH_PURITY_OXYGEN_f3,COAL_FLOW_f3,NAT_GAS _INJECTION_f3,HOT_BLAST_PRESSURE_f3,TOP_PRESS_f3,TOP_TEMP_HIGHEST_f3,Top_Gas_ETACO_f3,Selec_Top_Gas_CO_f3,Selec_Top_Gas_CO2_f3,Selec_Top_Gas_H2_f3,Selec_Top_Gas_N2_f3,SILICON_f3,SULFUR_f3,MANGANESE_f3,PHOSPHOROUS_f3,HOT_METAL_TEMP_f3,Slag_CaO_day_avg_f3,Slag_MgO_day_avg_f3,Slag_SiO2_f3,Slag_Al2O3_day_avg_f3,#14 WEST STOCKROD LEVEL,#14 EAST STOCKROD LEVEL,#14 RADAR ROD NORTH LEVEL,#14 RADAR ROD SOUTH LEVEL,Charges_per_hour_f3,BF14 material 1 weight,BF14 extra coke weight,BF14 material 2 weight,BF14 material 2 weight.1,BF14 material 3 weight,BF14 material 3 weight.1,BF14 material 4 weight,BF14 material 4 weight.1,BF14 material 5 weight,BF14 material 5 weight.1,BF14 material 6 weight,BF14 material 6 weight.1,BF14 material 7 weight,BF14 material 7 weight.1,BF14 material 8 weight,BF14 material 8 weight.1,BF14 material 9 weight,BF14 material 9 weight.1,BF14 material 10 weight,BF14 material 10 weight.1,NE_Uptake_f3,SE_Uptake_f3,NW_Uptake_f3,SW_Uptake_f3
0,0.0,44409.0,2809.0,182.0,1.152,0.042,0.577,0.077,0.003,0.020,0.052,0.002,0.036,2677.0,NaN,0.057639,44409.05764,NaN,4.692156,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2.0,0.0,11.4,602,2.75,165.0,2.12,1.0,2714.0,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,44409.0,2809.0,171.0,0.932,0.053,0.521,0.071,0.003,0.015,0.051,0.002,0.027,2729.0,NaN,0.086111,44409.08611,1.0,4.830898,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2.0,0.0,11.4,602,2.75,165.0,2.12,1.0,2714.0,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,44409.0,2809.0,122.0,1.059,0.043,0.540,0.076,0.003,0.020,0.051,0.002,0.032,2736.0,NaN,0.131250,44409.13125,NaN,4.804690,B,0.0,8/1/2021 0:00,00:59,00:55,03:23,2.0,0.0,11.4,602,2.75,165.0,2.12,1.0,2714.0,1.048,0.046,0.546,0.075,0.003,0.018,0.051,0.002,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,44409.0,2810.0,147.0,0.917,0.059,0.478,0.072,0.003,0.017,0.049,0.002,0.024,2557.0,NaN,0.145833,44409.14583,1.0,4.587464,B,0.0,8/1/2021 0:00,03:45,03:20,06:01,3.0,0.0,11.5,596,4.00,160.0,2.25,1.0,2677.0,0.906,0.057,0.488,0.072,0.003,0.016,0.049,0.002,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,44409.0,2810.0,190.0,1.023,0.054,0.508,0.072,0.003,0.012,